## Script for processing data with the UNET

In [83]:
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
from torchvision import models
from torch.nn.functional import relu
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import re
import mat73
from tqdm import tqdm
import pickle

In [3]:
model = torch.jit.load('Datasets/FinalModel/model_scripted.pt')
data = mat73.loadmat('Datasets/SpinningOPChangeNoisy/SpinningOPChangeNoisy.mat')
mask = sio.loadmat('mask.mat')['mask']
all_x = data['all_x']
all_y = data['all_y']
all_z = data['all_z']
locations = np.array([all_x, all_y, all_z])


In [4]:
test_X = torch.tensor(data['HbOvol'], dtype=torch.float32)

In [5]:
torch.max(test_X)

tensor(0.0613)

In [6]:
test_Y = np.zeros(test_X.shape)
for i in tqdm(range(test_X.shape[-1])):
    tmp = (test_X[..., i]-torch.min(test_X[...,i]))/(torch.max(test_X[...,i])-torch.min(test_X[...,i]))
    test_Y[...,i] = model(tmp.unsqueeze(0).unsqueeze(0)).squeeze().detach().numpy() * mask

100%|██████████| 756/756 [02:12<00:00,  5.72it/s]


In [7]:
data.keys()

dict_keys(['HbOvol', 'HbOvol_smoothed', 'all_amplitude', 'all_beta', 'all_dOD', 'all_dOD_noisy', 'all_fluctuate1', 'all_x', 'all_y', 'all_z', 'ground_truth'])

In [8]:
data['HbOvol_unet'] = test_Y
data['locations'] = locations

In [9]:
with open('Datasets/SpinningOPChangeNoisy/SpinningOPChangeNoisyWithUNETFinal.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
sio.savemat('Datasets/SpinningOPChangeNoisy/SpinningOPChangeNoisyWithUNETOnlyPicsFinal.mat', {'HbOvol_unet': test_Y, 'HbOvol': data['HbOvol'], 'HbOvol_smoothed': data['HbOvol_smoothed'], 'ground_truth' : data['ground_truth']})

In [11]:
del data

In [70]:
model = torch.jit.load('Datasets/FinalModel/model_scripted.pt')
data = mat73.loadmat('TestingData.mat')
mask = sio.loadmat('mask.mat')['mask']

In [76]:
test_X = torch.tensor(data['HbOvol'][:, :, :, :], dtype=torch.float32)

test_Y = np.zeros(test_X.shape)
for i in tqdm(range(test_X.shape[-1])):
    tmp = (test_X[:, :, :, i]-torch.min(test_X[...,i]))/(torch.max(test_X[...,i])-torch.min(test_X[...,i]))
    test_Y[:, :, :, i] = model(tmp.unsqueeze(0).unsqueeze(0)).squeeze().detach().numpy()

sio.savemat(r'Datasets/FinalModel/test_processed.mat', {'HbOvol_unet': test_Y, 'HbOvol': data['HbOvol'], 'HbOvol_smoothed': data['HbOvol_smoothed'], 'ground_truth' : data['ground_truth']})

100%|██████████| 100/100 [00:16<00:00,  6.01it/s]


In [73]:
print(data['ground_truth'])

[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 

In [26]:
model = torch.jit.load('Datasets/FinalModel/model_scripted.pt')
data = mat73.loadmat('CW1Neurodot.mat')
mask = sio.loadmat('mask.mat')['mask']

In [27]:
test_X = torch.tensor(data['HbOvol'], dtype=torch.float32)

In [28]:
test_Y = np.zeros(test_X.shape)
for i in tqdm(range(test_X.shape[-1])):
    tmp = (test_X[..., i]-torch.min(test_X[...,i]))/(torch.max(test_X[...,i])-torch.min(test_X[...,i]))
    test_Y[...,i] = model(tmp.unsqueeze(0).unsqueeze(0)).squeeze().detach().numpy() * mask

100%|██████████| 36/36 [00:07<00:00,  4.97it/s]


In [29]:
sio.savemat('Datasets/CW1/processed.mat', {'HbOvol_unet': test_Y, 'HbOvol': data['HbOvol'], 'HbOvol_smoothed': data['HbOvol_smoothed']})

In [36]:
A = sio.loadmat('CroppedWithjoenVtoggle/r2d.mat')

In [43]:
r2d = A['r2d']
r2d

array([[13.        ],
       [29.06888371],
       [46.87216658],
       ...,
       [46.87216658],
       [29.06888371],
       [13.        ]])

In [37]:
A

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Dec 31 13:27:37 2024',
 '__version__': '1.0',
 '__globals__': [],
 'r2d': array([[13.        ],
        [29.06888371],
        [46.87216658],
        ...,
        [46.87216658],
        [29.06888371],
        [13.        ]])}

In [42]:
r2d[np.where([1,0,1,0,1])]

array([[13.        ],
       [46.87216658],
       [29.06888371]])

In [ ]:
from FCNet import Linear
model = torch.load('Datasets/Training/FCNet/3D_UNet_trained3', map_location=torch.device('cpu'))
data = mat73.loadmat('Datasets/SpinningOPChangeNoisy/SpinningOPChangeNoisy.mat')
mask = sio.loadmat('mask.mat')['mask']
r2d = sio.loadmat('CroppedWithjoenVtoggle/r2d.mat')

In [82]:

test_X = torch.tensor(data['all_dOD_noisy'][np.nonzero(r2d['r2d']<=47)[1],:], dtype=torch.float32)
test_Y = np.zeros((80,32,64,test_X.shape[-1]))
for i in tqdm(range(test_X.shape[-1])):
    tmp = test_X[...,i]
    test_Y[..., i] = model(tmp.unsqueeze(0).unsqueeze(0)).squeeze().detach().numpy() * mask
sio.savemat('Datasets/SpinningOPChangeNoisy/processedFCNet.mat',
            {'HbOvol_unet': test_Y, 'HbOvol': data['HbOvol'], 'HbOvol_smoothed': data['HbOvol_smoothed']})

  0%|          | 0/756 [00:00<?, ?it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\joeol\.conda\envs\pythonProject\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\joeol\AppData\Local\Temp\ipykernel_5116\3650111440.py", line 5, in <module>
    test_Y[..., i] = model(tmp.unsqueeze(0).unsqueeze(0)).squeeze().detach().numpy() * mask
  File "C:\Users\joeol\.conda\envs\pythonProject\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "C:\Users\joeol\.conda\envs\pythonProject\lib\site-packages\torch\nn\modules\module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "C:\Joe Evans\Work\DOT\BrainDOT2\FCNet.py", line 30, in forward
  File "C:\Users\joeol\.conda\envs\pythonProject\lib\site-packages\torch\nn\modules\module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "C:\Users\joeol